In [ ]:
import arcpy
from flask import Flask, jsonify 
import psycopg2
from psycopg2 import sql

#Define database credentials
dbname = ""
dbuser = ""
dbpass = ""
dbhost = ""

#Make the Points for the Polygon
spatial_reference = arcpy.SpatialReference(4326)
point1 = arcpy.Point(2.2945, 48.8584)
point2 = arcpy.Point(-74.0445, 40.6892)
point3 = arcpy.Point(-72.5450,  -13.1631)

# Create an array using the points
myarray = arcpy.Array([point1, point2, point3])
#Create polygon using array and convert to WKT
mypolygon = arcpy.Polygon(myarray)
mypolygon_wkt = mypolygon.WKT

#Connect to Database
conn = psycopg2.connect(
    dbname=dbname,
    user=dbuser,
    password=dbpass,
    host= dbhost,
    port="5432"
    )

cur = conn.cursor()

#Create the table for geometry
table = sql.SQL("""
    CREATE TABLE lab1 (
    id SERIAL PRIMARY KEY,
   shape GEOMETRY(MultiPolygon, 4326)
    );
""")

# Insert polygon into the table
#Percent sign can be read within the query 
insert_query = sql.SQL("""
    INSERT INTO lab1 (shape)
    VALUES (ST_GeomFromText(%s, 4326));
""")

#Pulls query from variable and inserts into database. 
#Only need to execute table once
cur.execute(table)
cur.execute(insert_query, [mypolygon_wkt])
conn.commit()
cur.close()
conn.close()


In [ ]:
import os
from flask import Flask, jsonify 
import psycopg2
from psycopg2 import sql

#Name App
app = Flask(__name__)

#Define database credentials
dbname = "gis5572"
dbuser = "postgres"
dbpass = "mypassword123"
dbhost = "35.224.48.126"
    
#Connect to Database and Extract Geometry 
#App Route defines url
@app.route('/geojson_polygon.geojson')
def get_geojson():
    conn = psycopg2.connect(
        dbname=dbname,
        user=dbuser,
        password=dbpass, #Should be database_key if not connecting to local 
        host= dbhost,
        port="54
        )
    
    cur = conn.cursor()
    
#Query to return json with coordinates, feature type and CRS
    query = """
        SELECT 
            json_build_object(
                'type', 'FeatureCollection',
                'features', json_agg(
                    json_build_object(
                        'type', 'Feature',
                        'geometry', ST_AsGeoJSON(ST_SetSRID(shape, 4326))::json,
                        'properties', json_build_object()
                    )
                ),
                'crs', 
                json_build_object(
                    'type', 'name',
                    'properties', 
                    json_build_object(
                        'name', 'epsg:4326'
                    )
                )
            ) AS geojson
        FROM lab
    """
#Execure the query to return the GEOJSON
    cur.execute(query)
    geojson_data = cur.fetchone()[0]
    cur.close()
    conn.close()
#Return the geojson_data when the function is called
    return geojson_data

if __name__ == '__main__':
    app.run(debug=True, host="0.0.0.0", port=int(os.environ.get("PORT", 8080)))